In [12]:
import pandas as pd
import numpy as np
from neo4j import GraphDatabase
from matplotlib import pyplot as plt
from sklearn.metrics import auc
from sklearn.metrics import roc_curve
import ast

In [21]:
images_data = pd.read_csv("./objects_extract.csv")
images_data = images_data.dropna()
images_data["Description"] = images_data["Description"].apply(ast.literal_eval)
images_data.shape
print(images_data.head())

   Unnamed: 0           ImageID                         Description
0           0  00408efc3cabdc4e                              [Food]
1           1  0059eb01bba96297                              [Food]
2           2  0055444dd2ab3489                            [Person]
3           3  0028e126ab55ebfc  [Car, Man, Table, Clothing, Woman]
4           4  00336bd08f30f7f0  [Woman, Man, Human face, Clothing]


In [22]:
child_map = pd.read_csv("./child_map.csv")
child_map["Child"] = child_map["Child"].apply(ast.literal_eval)
child_map.shape

(76, 2)

In [23]:
# For every image description if it contains a child object change it to parent object
def get_parent(child):
    for index, row in child_map.iterrows():
        if child in row["Child"]:
            return row["Parent"]
    return child

for index, row in images_data.iterrows():
    description = row["Description"]
    new_description = []
    for obj in description:
        parent = get_parent(obj)
        if parent not in new_description:
            new_description.append(parent)
    images_data.at[index, "New Description"] = new_description
print(images_data.head())
    

   Unnamed: 0  ...                              New Description
0           0  ...                                       [Food]
1           1  ...                                       [Food]
2           2  ...                                     [Person]
3           3  ...  [Land vehicle, Person, Furniture, Clothing]
4           4  ...               [Person, Human body, Clothing]

[5 rows x 4 columns]


In [25]:
#get all children inside new description
def get_children(description):
    children = []
    children.extend(description)
    for obj in description:
        for index, row in child_map.iterrows():
            if obj == row["Parent"]:
                children.extend(row["Child"])
    return children

In [27]:
list_obj = ["Land vehicle", "Person", "Furniture", "Clothing"]
print(get_children(list_obj))

['Land vehicle', 'Person', 'Furniture', 'Clothing', 'Ambulance', 'Cart', 'Bus', 'Snowmobile', 'Golf cart', 'Motorcycle', 'Segway', 'Train', 'Truck', 'Unicycle', 'Car', 'Taxi', 'Man', 'Woman', 'Boy', 'Girl', 'Chair', 'Cabinetry', 'Desk', 'Wine rack', 'Couch', 'Wardrobe', 'Nightstand', 'Bookcase', 'Bed', 'Filing cabinet', 'Table', 'Chest of drawers', 'Cupboard', 'Bench', 'Drawer', 'Stool', 'Shelf', 'Bathroom cabinet', 'Closet', 'Shorts', 'Dress', 'Swimwear', 'Brassiere', 'Shirt', 'Coat', 'Suit', 'Skirt', 'Jacket', 'Fashion accessory', 'Swim cap', 'Trousers', 'Footwear', 'Sports uniform', 'Luggage and bags']


In [31]:
for index, row in images_data.iterrows():
    description = row["New Description"]
    all_children = get_children(description)
    images_data["All Children"].loc[index] = all_children
print(images_data.head())

C:\Users\Admin\AppData\Local\Temp\ipykernel_2072\635750318.py:4: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  images_data["All Children"].loc[index] = all_children
C:\Users\Admin\AppData\Local\Temp\ipykernel_2072\635750318.py:4: SettingWith

   Unnamed: 0  ...                                       All Children
0           0  ...  [Food, Fast food, Waffle, Pancake, Burrito, Sn...
1           1  ...  [Food, Fast food, Waffle, Pancake, Burrito, Sn...
2           2  ...                    [Person, Man, Woman, Boy, Girl]
3           3  ...  [Land vehicle, Person, Furniture, Clothing, Am...
4           4  ...  [Person, Human body, Clothing, Man, Woman, Boy...

[5 rows x 7 columns]


In [34]:
print(images_data.head())
images_data.drop(columns=["Description"], inplace=True)
images_data.drop(columns=["New Description"], inplace=True)
images_data.rename(columns={"All Children": "Description"}, inplace=True)
print(images_data.head())

   Unnamed: 0  ...                                       All Children
0           0  ...  [Food, Fast food, Waffle, Pancake, Burrito, Sn...
1           1  ...  [Food, Fast food, Waffle, Pancake, Burrito, Sn...
2           2  ...                    [Person, Man, Woman, Boy, Girl]
3           3  ...  [Land vehicle, Person, Furniture, Clothing, Am...
4           4  ...  [Person, Human body, Clothing, Man, Woman, Boy...

[5 rows x 7 columns]
   Unnamed: 0  ...                                        Description
0           0  ...  [Food, Fast food, Waffle, Pancake, Burrito, Sn...
1           1  ...  [Food, Fast food, Waffle, Pancake, Burrito, Sn...
2           2  ...                    [Person, Man, Woman, Boy, Girl]
3           3  ...  [Land vehicle, Person, Furniture, Clothing, Am...
4           4  ...  [Person, Human body, Clothing, Man, Woman, Boy...

[5 rows x 5 columns]


In [38]:
images_data.drop(columns=["Final Description"], inplace=True)
images_data.drop(columns=["Children"], inplace=True)

In [39]:
images_data.to_csv("objects_extract_fixed.csv", index=False)